In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

endpoint = "https://lnguyentuan-azure-openai.openai.azure.com/"
model_name = "gpt-4o"
deployment = "gpt-4o"

subscription_key = os.getenv("KEY")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

Paris is a city rich with history, culture, and world-famous attractions, so you won't run out of things to see and do. Here’s a list of must-see places to help you plan your trip:

### **Iconic Landmarks:**
1. **Eiffel Tower**  
   - No trip to Paris is complete without visiting this iconic structure. You can take photos from below, ascend to various levels for breathtaking views, or enjoy a meal or drink at its restaurants.

2. **Louvre Museum**  
   - The world’s largest art museum and home to famous works like the **Mona Lisa**, **Venus de Milo**, and thousands of other treasures. Even the glass pyramid outside is worth seeing.

3. **Notre-Dame Cathedral** (*Currently under renovation until 2024 due to fire damage*)  
   - A masterpiece of Gothic architecture known for its stunning facade, rose windows, and historical significance. Nearby, you can stroll along the Seine or explore Île de la Cité.

4. **Arc de Triomphe & Champs-Élysées**  
   - Marvel at this triumphal arch honoring